In [1]:
#参考https://github.com/farizrahman4u/seq2seq
import numpy as np
import pandas as pd

PARTICIPANT_ID='Sub01'

df_csv=pd.read_csv('data/Provo_Corpus-Eyetracking_Data(sub1-10).csv',usecols=['Participant_ID','Text_ID','Word_Cleaned','IA_SKIP'])
df_csv=df_csv[(df_csv['Participant_ID']==PARTICIPANT_ID)]
df_csv=df_csv.drop(['Participant_ID'],axis=1) #删除列，轴为1
df_csv=df_csv.fillna(' ')
list_data=[]
for j in range(1,56): #句子1-55
    df=df_csv[(df_csv['Text_ID']==j)][['Word_Cleaned','IA_SKIP']] #只保留'Word_Cleaned','IA_SKIP'两列
    mylist = np.array(df).tolist()
    list_data.append(mylist)

texts=[[j[0] for j in i]for i in list_data]
labels=[[j[1] for j in i]for i in list_data]
#print(labels)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS = 2000 #字典大小
MAX_SEQUENCE_LENGTH = max(len(s) for s in texts)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
MAX_NB_WORDS = len(word_index)+1 #重新修正字典大小
#print(word_index)
x = pad_sequences(sequences, MAX_SEQUENCE_LENGTH)  # left padding
x = np.expand_dims(x, 2) 
print(x)
print(x.shape)
y = pad_sequences(labels, MAX_SEQUENCE_LENGTH, value=-1)  # left padding
y = np.expand_dims(y, 2) 
#这是因为crf层期望标签具有不同的形状。
#通常，我们的标签的shape为(num_samples,max_length)，但crf层需要的shape为(num_samples,max_length,1)。
#解决方法：将标签扩一维
#https://blog.csdn.net/qq_31456593/article/details/89578018

#用Dense做激活层的时候，仍然需要这个expand_dims。

print(y)
print(y.shape)
train_x=x[0:45]
train_y=y[0:45]
test_x=x[45:]
test_y=y[45:]

Using TensorFlow backend.


[[[   0]
  [   0]
  [   0]
  ...
  [   3]
  [ 340]
  [   6]]

 [[   0]
  [ 160]
  [ 161]
  ...
  [  15]
  [ 363]
  [   6]]

 [[   0]
  [   0]
  [ 364]
  ...
  [ 377]
  [ 378]
  [   6]]

 ...

 [[   0]
  [   0]
  [   0]
  ...
  [  32]
  [  54]
  [   6]]

 [[   0]
  [   0]
  [   0]
  ...
  [1172]
  [ 120]
  [   6]]

 [[ 184]
  [1173]
  [1174]
  ...
  [1192]
  [   6]
  [   6]]]
(55, 60, 1)
[[[-1]
  [-1]
  [-1]
  ...
  [ 1]
  [ 0]
  [ 0]]

 [[-1]
  [ 0]
  [ 0]
  ...
  [ 1]
  [ 0]
  [ 0]]

 [[-1]
  [-1]
  [ 0]
  ...
  [ 0]
  [ 0]
  [ 0]]

 ...

 [[-1]
  [-1]
  [-1]
  ...
  [ 1]
  [ 0]
  [ 0]]

 [[-1]
  [-1]
  [-1]
  ...
  [ 0]
  [ 1]
  [ 0]]

 [[ 1]
  [ 1]
  [ 1]
  ...
  [ 0]
  [ 1]
  [ 0]]]
(55, 60, 1)


In [3]:
input_length = MAX_SEQUENCE_LENGTH
input_dim = 1

output_length = MAX_SEQUENCE_LENGTH
output_dim = 1

samples = 55
hidden_dim = 24

from seq2seq import SimpleSeq2Seq, Seq2Seq, AttentionSeq2Seq
model=AttentionSeq2Seq(output_dim=output_dim, hidden_dim=hidden_dim, output_length=output_length, input_shape=(input_length, input_dim), depth=3)
model.summary()
model.compile(loss='mse', optimizer='rmsprop',metrics=['acc'])
model.fit(x, y,batch_size=4,epochs=15)
model.save('model/crf.h5')

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 1)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 60, 24)       23808       input_1[0][0]                    
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
_____________________________________